In [1]:
import pandas as pd

In [2]:
#file to load
file_to_load="Resources/purchase_data.csv"

#read file and store into Pandas Data Frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#display the total number of unique players
total_players = len(purchase_data["SN"].unique())
total_players


576

In [4]:
#display the total number of unique in game items
unique_items = len(purchase_data["Item ID"].unique())
unique_items

179

In [5]:
#calculate average price per item
price_average = purchase_data["Price"].mean()
price_average

3.0509871794871795

In [6]:
#total revenue from all in game purchases
total_revenue=purchase_data["Price"].sum()
total_revenue


2379.77

In [7]:
#total number of purchases 
total_purchases=purchase_data["Purchase ID"].count()
total_purchases

780

In [8]:
#Summary Data Frame/cleaned up for organization 
summary_df_1=pd.DataFrame([{"Number of Unique Items": unique_items,
                           "Average Price": price_average,
                           "Number of Purchases":total_purchases,
                           "Total Revenue":total_revenue}])
summary_df_1

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


In [9]:
#calculate information for gender demographics
gender_demo=purchase_data["Gender"].value_counts()
gender_percent=round(purchase_data["Gender"].value_counts("Male"),2)
print(gender_demo)
print()
gender_percent


Male                     652
Female                   113
Other / Non-Disclosed     15
Name: Gender, dtype: int64



Male                     0.84
Female                   0.14
Other / Non-Disclosed    0.02
Name: Gender, dtype: float64

In [10]:
#purchasing analysis by gender 
gender_purchases = purchase_data[["Price", "Gender"]]

gender_purchases = purchase_data.groupby(["Gender"])
gender_purchases = gender_purchases.count()
gender_purchases

#manipulates df below into 2 column gender & price
gender_price=gender_purchases[["Price"]]
gender_price.head()



,Price
Gender,
Female,113
Male,652
Other / Non-Disclosed,15


In [11]:
#Age Demographics

#Find the min and max ages of the players to aid in creating bins
print(purchase_data["Age"].max())
print(purchase_data["Age"].min())


45
7


In [13]:
bins= [0,9,14,19,24,29,34,39,100]
group_labels=["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

#Slice the data and place it in bins 
pd.cut(purchase_data["Age"],bins, labels=group_labels).head()


0    20-24
1      40+
2    20-24
3    20-24
4    20-24
Name: Age, dtype: category
Categories (8, object): [<10 < 10-14 < 15-19 < 20-24 < 25-29 < 30-34 < 35-39 < 40+]

In [25]:
age_group=purchase_data.groupby("Age")
print(age_group["Age"].count())

Age
7      9
8      8
9      6
10     9
11     7
12     6
13     4
14     2
15    35
16    30
17    22
18    26
19    23
20    99
21    62
22    70
23    67
24    67
25    59
26    14
27    10
28     5
29    13
30    35
31     7
32     8
33    14
34     9
35    14
36     5
37     7
38     9
39     6
40     6
41     2
42     1
43     1
44     2
45     1
Name: Age, dtype: int64
